# Keyphrase similality

Date: 2024/06/15

The ChromaDB's default embedder "sentence-transformers/all-MiniLM-L6-v2"

Choose a multi-lingual one for Japanese:

- https://docs.trychroma.com/guides/embeddings
- https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

In [1]:
import spacy
import textacy
from textacy import extract

In [2]:
# Text generated with ChatGPT
text = open("beatles.txt").read()

In [3]:
nlp = spacy.load("ja_core_news_lg")
doc = nlp(text)

/Users/shiny/Documents/GitHub/learning-langchain/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
sents = [sent.text.replace('・', ' ') for sent in doc.sents]

In [5]:
en = textacy.load_spacy_lang("ja_core_news_lg")

In [14]:
import re
kps = []

def normalize(x):
    if re.search(r'[ぁ-ん]+|[ァ-ヴー]+|[一-龠]+', x):
        return x.replace(' ', '')
    else:
        return x.lower()
        
for sent in sents:
    doc = textacy.make_spacy_doc(sent, lang=en)
    kps_textrank = [normalize(kps) for kps, _ in extract.keyterms.textrank(doc, normalize="lemma", topn=10)]
    #print(kps_textrank)
    kps.extend(kps_textrank)

In [15]:
kps

['ロックバンド',
 '年代',
 'イギリス',
 '人気',
 '世界',
 'ビートルズ',
 'ジョンレノン',
 'リンゴスター',
 'ポールマッカートニー',
 'ジョージハリスン',
 'ringo starr',
 'paul mccartney',
 'john lennon',
 'george harrison',
 '人',
 'メンバー',
 '音楽史',
 '多様性',
 '革新性',
 'ハードロック',
 'サイケデリックロック',
 '大きな影響',
 '多岐',
 'ポップ',
 '結成',
 '活動',
 'ビートルズ',
 '初期',
 'リヴァプール',
 'ピートベスト',
 'レノン',
 'メンバー',
 'ドラマー',
 'マッカートニー',
 'ハリスン',
 '当初',
 '演奏活動',
 'ハンブルク',
 '過酷',
 'スキル',
 'クラブ',
 '結束力',
 '期間',
 '音楽',
 'バンド',
 '成長',
 'リンゴスター',
 'ジョージマーティン',
 '年',
 'バンド',
 'ベスト',
 '代わり',
 '勧め',
 'プロデューサー',
 'ラヴミードゥ',
 'love me do',
 'デビューシングル',
 'ブレイクスルー',
 '始まり',
 '年',
 'ビートルズ',
 '月',
 '成功',
 'フロムミートゥユー',
 'from me to you',
 'プリーズプリーズミー',
 'please please me',
 'シーラヴズユー',
 'she loves you',
 'イギリス国内',
 'ヒット曲',
 '年',
 '爆発',
 'エドサリヴァンショー',
 'ビートルマニア',
 '進出',
 'アメリカ',
 '出演',
 '年',
 'ビートルズ',
 'アメリカ国内',
 '一大センセーション',
 'ビートルマニア',
 'パフォーマンス',
 '現象',
 'i want to hold your hand',
 'ビルボードチャート',
 'トップスター',
 '抱く',
 'シングル',
 'アメリカ',
 '位',
 '座',
 '大きな進化',
 '年',
 '音楽',
 'ビートルズ',
 '高い評価',
 '音

In [8]:
import chromadb
from chromadb.utils import embedding_functions

MODEL_NAME = "distiluse-base-multilingual-cased-v2"
#MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2" ==> Does not work for Japanese lang
#MODEL_NAME = "paraphrase-multilingual-mpnet-base-v2" ==> Does not work for Japanese lang


multi = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=MODEL_NAME)

chroma_client = chromadb.Client()

/Users/shiny/Documents/GitHub/learning-langchain/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
collection = chroma_client.create_collection(name="collection-beatles3", embedding_function=multi)

In [10]:
ids = [str(i) for i in range(len(kps))]

In [11]:
collection.add(
    documents=kps,
    ids=ids
)

In [30]:
QUERY_TEXTS = ["ジョンレノン", "ジョン", "ポール", "ジョージ", "リンゴ", "影響力", "サイケデリック", "インド", "移住", "成功",
               "レジェンド", "レガシー", "Sgt pepper's lonely hearts club band", "Popular songs"]

In [31]:
results = collection.query(
    query_texts=QUERY_TEXTS,
    n_results=5 # how many results to retur
)

ids = results["ids"]
distances = results["distances"]
documents = results["documents"]

THRESHOLD = 0.27

for idx, dist in enumerate(results["distances"]):
    for i, d in enumerate(dist):
        if d <= THRESHOLD:
            print(QUERY_TEXTS[idx], ids[idx][i], documents[idx][i], f"{distances[idx][i]:.3f}")

ジョンレノン 6 ジョンレノン 0.000
ジョンレノン 202 ジョンレノン 0.000
ジョンレノン 12 JOHN lennon 0.193
ポール 23 ポップ 0.249
ジョージ 9 ジョージハリスン 0.143
ジョージ 201 ジョージハリスン 0.143
ジョージ 47 ジョージマーティン 0.189
ジョージ 60 ビートルズ 0.244
ジョージ 170 ビートルズ 0.244
リンゴ 210 レガシー 0.252
リンゴ 117 アルバム 0.263
リンゴ 260 アルバム 0.263
リンゴ 106 アルバム 0.263
リンゴ 176 アルバム 0.263
影響力 280 影響力 0.000
影響力 194 影響力 0.000
影響力 298 影響力 0.000
影響力 239 影響 0.045
影響力 208 影響 0.045
サイケデリック 111 サイケデリック 0.000
サイケデリック 39 スキル 0.105
サイケデリック 250 ビートルズ 0.125
サイケデリック 96 ビートルズ 0.125
サイケデリック 60 ビートルズ 0.125
インド 138 インド 0.000
移住 75 進出 0.236
成功 62 成功 0.000
成功 204 成功 0.000
成功 123 完成度 0.245
レジェンド 210 レガシー 0.041
レジェンド 100 ラバーソウル 0.064
レジェンド 124 名盤 0.088
レジェンド 151 ジャンル 0.100
レジェンド 117 アルバム 0.100
レガシー 210 レガシー 0.000
レガシー 100 ラバーソウル 0.072
レガシー 151 ジャンル 0.113
レガシー 124 名盤 0.115
レガシー 277 リスナー 0.120
Sgt pepper's lonely hearts club band 115 Pepper's lonely hearts CLUB band 0.167


## Conclusion

Not so bad.